In [ ]:
import pathlib
import pandas as pd
import numpy as np

# Path
BASE_PATH = pathlib.Path("testing.ipynb").parent.resolve()
DATA_PATH = BASE_PATH.joinpath("data").resolve()

attendees = pd.read_csv(DATA_PATH.joinpath("Q3-2019 Attendees.csv"))

In [ ]:
attendees.head()

In [ ]:
q3_analysis_work = attendees.copy()

q3_analysis_work = q3_analysis_work.rename(columns = {'Order #':'order#', 'Ticket Type':'ticket_type', 'Attendee #':'attendee#', 'Gender': 'gender', 
                                                      'City': 'city', 'Zip Code': 'zipcode', 'Country': 'country', 
                                                      'Dietary Restrictions': 'dietary_restrictions', 'Technical Expertise': 'technical_expertise',
                                                      'Analytics Experience': 'analytics_experience', 'Job Function': 'job_function', 'Industry': 'industry',
                                                      'What topics would you like to hear about?': 'topics_like_hear',
                                                      'Are you interested in being recruited by sponsoring companies?': 'interested_being_recruited',
                                                      'Do you plan on attending the opening reception on Wednesday, June 3rd, 2020?': 'plan_attending_reception',
                                                      'Did someone refer you to this conference? If so, please list their name.': 'someone_referred_ifso_name',
                                                      'Additional Terms': 'additional_terms', 'Job Title': 'job_title', 'Company': 'company'})
col = ['ticket_type', 'attendee#', 'gender', 'city', 'zipcode',
       'country', 'dietary_restrictions', 'technical_expertise',
       'analytics_experience', 'job_function', 'industry', 'topics_like_hear',
       'interested_being_recruited', 'plan_attending_reception',
       'someone_referred_ifso_name', 'additional_terms', 'job_title',
       'company']
for i in col:
    q3_analysis_work[col] = q3_analysis_work[col].replace(np.nan,'No Answer')

In [ ]:
q3_analysis_work.head()


In [ ]:

import numpy as np
import io
import pandas as pd
from stop_words import get_stop_words
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
import gensim
import matplotlib.pyplot as plt
from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS
from PIL import Image
import PIL.ImageOps
import random
from wordcloud import ImageColorGenerator

In [ ]:

# Listed of stop-words found at
# https://pypi.python.org/pypi/stop-words

stop_words = get_stop_words('en')

# Clean up text
tokenizer = RegexpTokenizer(r'\w+')
texts = []
for i in q3_analysis_work['job_title'].dropna(how='all'):
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [i for i in tokens if not i in stop_words]
    longer_tokens = [i for i in stopped_tokens if len(i) > 2]
    texts.append(longer_tokens)

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Run tf-idf
tfidf = models.TfidfModel(corpus)

##WIA

# Based off example:
# https://github.com/amueller/word_cloud/blob/master/examples/a_new_hope.py

def red_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(24, 99%%, %d%%)" % random.randint(40, 70)

ASSETS_PATH = BASE_PATH.joinpath("assets").resolve()

wia_mask = np.array(Image.open(ASSETS_PATH.joinpath("WIA_logo.jpg")))
#top_words = np.sort(np.array(tfidf[corpus[57]],dtype = [('word',int), ('score',float)]),order='score')[::-1]

wc = WordCloud(background_color="white",mask=wia_mask,random_state=5,max_words=2000).generate(str(texts))
#.fit_words(dict([(dictionary[word],score) for word,score in top_words])))

plt.imshow(wc.recolor(color_func=red_color_func, random_state=5),interpolation='bilinear')
plt.axis("off")
wc.to_file("wia_title_wordcloud.png")

In [ ]:
import nltk
import re
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from operator import itemgetter
WNL = nltk.WordNetLemmatizer()

q3_analysis_work['job_title'] = q3_analysis_work['job_title'].str.replace(' / ', '')
q3_analysis_work['job_title'] = q3_analysis_work['job_title'].str.replace('Sr', 'Senior')

df2 = q3_analysis_work['job_title'].dropna(how='all').astype(str)
df2 = " ".join(job_title for job_title in df2)

stopwords = set(STOPWORDS)
stopwords.update(["I", "on", "and", "good", "etc", "make", "better", "depending"])

# Generate a word cloud image
text_content = [WNL.lemmatize(str(df2)) for t in df2]

tokens = nltk.word_tokenize(str(df2))
text = nltk.Text(tokens)

# Remove extra chars and remove stop words.
text_content = [''.join(re.split("[ .,;:!?‘’``''@#$%^_&*()<>{}~\n\t\\\-]", word)) for word in text]
text_content = [word for word in text_content if word not in stopwords]
 
# After the punctuation above is removed it still leaves empty entries in the list.
# Remove any entries where the len is zero.
text_content = [s for s in text_content if len(s) != 0]
 
# Best to get the lemmas of each word to reduce the number of similar words
# on the word cloud. The default lemmatize method is noun, but this could be
# expanded.
# ex: The lemma of 'characters' is 'character'.
text_content = [WNL.lemmatize(t) for t in text_content]
 
# setup and score the bigrams using the raw frequency.
finder = BigramCollocationFinder.from_words(text_content)
bigram_measures = BigramAssocMeasures()
scored = finder.score_ngrams(bigram_measures.raw_freq)



# By default finder.score_ngrams is sorted, however don't rely on this default behavior.
# Sort highest to lowest based on the score.
scoredList = sorted(scored, key=itemgetter(1), reverse=True)
 
# word_dict is the dictionary we'll use for the word cloud.
# Load dictionary with the FOR loop below.
# The dictionary will look like this with the bigram and the score from above.
# word_dict = {'bigram A': 0.000697411,
#             'bigram B': 0.000524882}
 
word_dict = {}
 
listLen = len(scoredList)
 
# Get the bigram and make a contiguous string for the dictionary key. 
# Set the key to the scored value. 
for i in range(listLen):
    word_dict[' '.join(scoredList[i][0])] = scoredList[i][1]
 
word_dict.update({'Analyst Data':0,'Data Analytics':0})

def red_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(0, 99%%, %d%%)" % random.randint(40, 70)

#ASSETS_PATH = BASE_PATH.joinpath("assets").resolve()

#wia_mask = np.array(Image.open(ASSETS_PATH.joinpath("WIA_logo_heavy.jpg")))


# -----
 
# Set word cloud params and instantiate the word cloud.
# The height and width only affect the output image file.
WC_height = 500
WC_width = 1000
WC_max_words = 100
 
wc = WordCloud(max_words=WC_max_words, height=WC_height, width=WC_width,stopwords=stopwords, background_color="white")
 
wc.generate_from_frequencies(word_dict)
 
plt.imshow(wc.recolor(color_func=red_color_func, random_state=5), interpolation='bilinear')
plt.axis("off")
plt.show()
wc.to_file("wia_title_wordcloud.png")

In [ ]:
import nltk
import re
from nltk.collocations import QuadgramAssocMeasures,QuadgramCollocationFinder
from operator import itemgetter
WNL = nltk.WordNetLemmatizer()

df2 = q3_analysis_work['company'].dropna(how='all').astype(str)

df2.replace('CMM','CoverMyMeds',inplace=True)
df2.replace('The James Cancer Hospital/OSUWMC','The James Cancer Hospital',inplace=True)
df2.replace('James Cancer Hospital','The James Cancer Hospital',inplace=True)
df2.replace('OSU - The James','The James Cancer Hospital',inplace=True)
df2.replace('OSUCCC - The James','The James Cancer Hospital',inplace=True)
df2.replace('Ohio State University Wexner Medical Center The James','The Ohio State University Wexner Medical Center',inplace=True)

#df2 = "_".join(company_title for company_title in df2)

stopwords = set(STOPWORDS)

# Generate a word cloud image
text_content = [WNL.lemmatize(str(df2)) for t in df2]

tokens = nltk.word_tokenize(str(df2))
text = nltk.Text(tokens)

# Remove extra chars and remove stop words.
text_content = [''.join(re.split("[ .,;:!?‘’``''@#$%^_&*()<>{}~\n\t\\\-]", word)) for word in text]
text_content = [word for word in text_content if word not in stopwords]
 
# After the punctuation above is removed it still leaves empty entries in the list.
# Remove any entries where the len is zero.
text_content = [s for s in text_content if len(s) != 0]
 
# Best to get the lemmas of each word to reduce the number of similar words
# on the word cloud. The default lemmatize method is noun, but this could be
# expanded.
# ex: The lemma of 'characters' is 'character'.
#text_content = [WNL.lemmatize(t) for t in text_content]
 
# setup and score the bigrams using the raw frequency.
finder = QuadgramCollocationFinder.from_words(text_content)
quadgram_measures = QuadgramAssocMeasures()
scored = finder.score_ngrams(quadgram_measures.raw_freq)



# By default finder.score_ngrams is sorted, however don't rely on this default behavior.
# Sort highest to lowest based on the score.
scoredList = sorted(scored, key=itemgetter(1), reverse=True)
 
# word_dict is the dictionary we'll use for the word cloud.
# Load dictionary with the FOR loop below.
# The dictionary will look like this with the bigram and the score from above.
# word_dict = {'bigram A': 0.000697411,
#             'bigram B': 0.000524882}
 
word_dict = {}
 
listLen = len(scoredList)
 
# Get the bigram and make a contiguous string for the dictionary key. 
# Set the key to the scored value. 
for i in range(listLen):
    word_dict[' '.join(scoredList[i][0])] = scoredList[i][1]

def blue_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(240, 99%%, %d%%)" % random.randint(20, 80)

#ASSETS_PATH = BASE_PATH.joinpath("assets").resolve()

#wia_mask = np.array(Image.open(ASSETS_PATH.joinpath("WIA_logo_heavy.jpg")))


In [ ]:
# -----
 
# Set word cloud params and instantiate the word cloud.
# The height and width only affect the output image file.
WC_height = 500
WC_width = 1000
WC_max_words = 100
 
wc = WordCloud(max_words=WC_max_words, height=WC_height, width=WC_width, background_color="white",prefer_horizontal=0.7)
 
wc.generate_from_frequencies(dict(df2.value_counts()))

plt.imshow(wc.recolor(color_func=blue_color_func, random_state=5),interpolation='bilinear')
plt.axis("off")
plt.show()
wc.to_file("wia_company_wordcloud.png")

In [ ]:
q3_analysis_work.technical_expertise.unique()

In [ ]:
import pandas as pd
from plotly import express as px
import plotly

df = q3_analysis_work.technical_expertise.value_counts().to_frame().reset_index().rename(columns={"index":"Technical Expertise","technical_expertise":"Frequency"})

p = px.histogram(
    df,
    x='Technical Expertise',
    y='Frequency',
    histnorm="percent",
    histfunc="sum",
    color_discrete_map={'':['gray','gray','indianred','gray','gray']},
    labels={'Frequency':r'of Attendees (%)'},
    category_orders={'Technical Expertise':['Basic Knowledge','Novice','Intermediate','Advanced','Expert']},
    title="Reported Technical Expertise")

import json
json.dumps(p, cls=plotly.utils.PlotlyJSONEncoder)

In [ ]:
df = q3_analysis_work.analytics_experience.value_counts().to_frame().reset_index().rename(columns={"index":"Analytics Experience","analytics_experience":"Frequency"})
px.histogram(
    df,
    x='Analytics Experience',
    y='Frequency',
    histnorm="percent",
    histfunc="sum",
    color_discrete_map={'':['gray','indianred','gray','gray']},
    labels={'Frequency':r'of Attendees (%)'},
    category_orders={'Analytics Experience':['< 1 Year','2-4 Years','5-9 Years','10+ Years']},
    title="Reported Experience in Analytics")

In [ ]:
from plotly import express as px
import pandas as pd
from PIL import Image

px.bar_polar(
    q3_analysis_work.industry.value_counts().to_frame() .reset_index().rename(columns={"index":"Industry","industry":"Frequency"}),
    r='Frequency',
    theta='Industry',
    barnorm='percent',
    color_discrete_map={'':px.colors.sequential.Sunsetdark},
    width=800,
    height=550,
    start_angle=95,
    title='Industry of Attendees (%)'
    ).update(layout=dict(images=[dict(
        source=Image.open(pathlib.Path("testing.ipynb").parent.resolve().joinpath("assets").joinpath("WIA_logo.jpg")),
        xref="paper", yref="paper",
        x=0.461, y=0.452,
        sizex=0.1, sizey=0.1,
        xanchor="left", yanchor="bottom"
    )],
    title=dict(
        x=0.5
        ),
        font_size=13,
        polar=dict(
                sector=[89,435],
                hole=0.15,
                bargap=0.3,
                radialaxis=dict(
                    gridcolor="lightgray",
                    showline=False,
                    tickangle=90,
                    ticksuffix="%"
                ),
                angularaxis=dict(
                        gridcolor="gray"
                    ),
                    gridshape="circular",
                    bgcolor="white"
                    )
                )
            )

In [ ]:
import pandas as pd 
locations = pd.read_csv(DATA_PATH.joinpath("q3_location.csv"))
locations.State.value_counts().to_frame().reset_index()

In [ ]:
import plotly.graph_objects as go

df = locations.State.value_counts().to_frame().reset_index()
df['State'] = df['State'].astype(float) / df['State'].astype(float).sum() * 100

fig = go.Figure(data=go.Choropleth(
    locations=df['index'], # Spatial coordinates
    z = df['State'].astype(int), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = r"% of Attendees",
    colorbar_ticksuffix="%",
))

fig.update_layout(
    title_text = 'Attendees by State',
    geo_scope='north america', # limit map scope to NA
    geo = dict(
        showframe=False,
        projection=go.layout.geo.Projection(type = 'mercator'),
    )
)

fig.show()
# px.set_mapbox_access_token("68f2625c-b463-45ae-8ec3-7f06f83b2073")
# px.choropleth(
#     data_frame=locations.State.value_counts().to_frame().reset_index(),
#     locations="index",
#     locationmode="USA-states",
#     scope="north america",
#     projection="mercator",
#     color="State",
#     color_continuous_scale=px.colors.sequential.RdBu
# )




In [ ]:
import plotly.graph_objects as go
import plotly.figure_factory as ff
import pandas as pd
import numpy as np

df = locations.State.value_counts().to_frame().reset_index()
df['State'] = df['State'].astype(float) / df['State'].astype(float).sum() * 100

colors = ['rgb(239,243,255)', 'rgb(189,215,231)', 'rgb(107,174,214)', 'rgb(33,113,181)']

fig = go.Figure()

ff.create_choropleth(

)


#scatter chart of ohio
# fig.add_trace(go.Scattergeo(
#     lon = df_month['Lon'],
#     lat = df_month['Lat'],
#     text = df_month['Value'],
#     name = months[i],
#     marker = dict(
#         size=df_month['Value']/50,
#         color=colors[i-6],
#         line_width=0)
#     )
# )

df_sept = df.query('Month == 9')
fig.data[0].update(text = df_sept['Value'].map('{:.0f}'.format).astype(str)+' '+\
                        df_sept['Country'],
                     mode = 'markers+text',
                     textposition = 'bottom center')


fig.add_trace(go.Choropleth(
        locationmode='country names',
        locations=df_sept['Country'],
        z=df_sept['Value'],
        text=df_sept['Country'],
        colorscale = [[0,'rgb(0, 0, 0)'],[1,'rgb(0, 0, 0)']],
        autocolorscale = False,
        showscale = False,
        geo = 'geo2'
    ))
fig.add_trace(go.Scattergeo(
        lon = [21.0936],
        lat = [7.1881],
        text = ['Africa'],
        mode = 'text',
        showlegend = False,
        geo = 'geo2'
    ))

fig.update_layout(
    title_text = 'Ebola cases reported by month in West Africa 2014<br> \
Source: <a href="https://data.hdx.rwlabs.org/dataset/rowca-ebola-cases">\
HDX</a>',
    geo = dict(
        resolution=50,
        scope='africa',
        showframe=False,
        showcoastlines=True,
        showland=True,
        landcolor="lightgray",
        countrycolor="white" ,
        coastlinecolor="white",
        projection_type='equirectangular',
        lonaxis_range=[ -15.0, -5.0],
        lataxis_range=[ 0.0, 12.0],
        domain = dict(x=[0, 1], y=[ 0, 1])
    ),
    geo2 = dict(
        scope='africa',
        showframe=False,
        showland=True,
        landcolor="lightgray",
        showcountries=False,
        domain=dict(x=[ 0, 0.6], y=[ 0, 0.6]),
        bgcolor='rgba(255, 255, 255, 0.0)',
    ),
    legend_traceorder = 'reversed'
)

fig.show()

In [ ]:
df2 = locations[locations.State == "OH"]['FIPS'].value_counts().to_frame().reset_index().rename(columns={'FIPS':'Percent','index':'FIPS'})
df2['Percent'] = df2['Percent'] / locations.shape[0] * 100

colorscale = [
    'rgb(193, 193, 193)',
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(65, 53, 132)'
]

#endpts = list(np.linspace(1,60, len(colorscale) - 1))
endpts = list(np.mgrid[min(values):max(values):4j])
fips = df2['FIPS'].tolist()
values = df2['Percent'].tolist()


fig = ff.create_choropleth(
    fips=fips, values=values, scope=['Ohio'],
    binning_endpoints=endpts, colorscale=colorscale,
    show_state_data=True,
    show_hover=True,
    simplify_county=False,
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    round_legend_values=True,
    asp = 2.9,
)
fig.layout.template = None
fig.show()

In [ ]:
df2 = locations[locations.State == "OH"]['FIPS'].value_counts().to_frame().reset_index().rename(columns={'FIPS':'Percent','index':'FIPS'})
df2['Percent'] = df2['Percent'] / locations.shape[0] * 100
df2

In [ ]:
df2 = locations[locations.State == "OH"]['City'].value_counts().to_frame().reset_index().rename(columns={'FIPS':'Percent','index':'FIPS'})
df2['Percent'] = df2['Percent'] / locations.shape[0] * 100
df2

In [ ]:
from geopy.geocoders import MapBox
loc = MapBox("pk.eyJ1IjoibXdhbGtzIiwiYSI6ImNrMDZzdHowdTAxcTAzZG9nMWM5ZW9teXUifQ.rSvhyBymD4tJEPHwJxu0Fw")
latitudes = list()
longitudes = list()
for row in locations.iterrows():
    locdone = loc.geocode(query= dict({'city':str(row[1][0]),'state':str(row[1][1]),'country':str(row[1][2])}))
    latitudes.append(locdone.latitude)
    longitudes.append(locdone.longitude)
    

In [ ]:
locations.columns

In [ ]:
locations['lat'] = [float(i) for i in latitudes]
locations['lon'] = [float(i) for i in longitudes]
#latitudes = [float(i) for i in latitudes]
#longitudes = [float(i) for i in longitudes]

In [ ]:
lonlat = pd.concat([locations.lon.value_counts().to_frame().reset_index().rename(columns={'index':'lon','lon':'count'}).drop('count',axis=1),locations.lat.value_counts().to_frame().reset_index().rename(columns={'index':'lat','lat':'percent'})],axis=1)
lonlat['percent'] = lonlat['percent'] / lonlat['percent'].sum()

In [ ]:
px.scatter_geo(
    loc_final,
    lat='lat',
    lon='lon',
    size='percent',
    size_max=40,
    opacity=0.7,
    hover_name='city',
    scope='usa',
    title='Attendee Proportions by City',
    color_discrete_map={'':'orange'},
    width=700,
    height=500,
).update(layout=dict(title=dict(x=0.5),geo=dict(projection_type='albers usa',subunitcolor='gray',subunitwidth=0.5,countrycolor='gray')))

In [ ]:
l = []
for i in locations[['lon','lat']].iterrows():
     l.append(i[1][0] in longitudes and i[1][1] in latitudes)

In [ ]:
loc_final

In [ ]:
newdf = pd.merge(pd.merge(left=lonlat,right=locations[['City','lon','lat']],on='lat',how='left').drop_duplicates(),locations[['City','lon','lat']],left_on='lon_x',right_on='lon',how='outer')
newdf = newdf[['City_y','lon_x','lat_y','percent']]#.rename{columns={'city','lon','lat'}}
newdf.columns = ['city','lon','lat','percent']
loc_final = newdf.drop_duplicates().reset_index().drop('index',axis=1)

In [ ]:
locations[locations.City == 'Cleveland']

In [ ]:
loc_final.to_csv("shortcut_loc.csv")